In [0]:
%run ../../_utils

In [0]:
from pyspark.sql.functions import col, sum, count


# Camada GOLD

Na camada gold, as limpezas e ajustes já foram feitos, então essa camada é responsável por aplicar regras de negócio, agregações e junções de dados que convirjam para analises.

Essa é uma tabela sumarizada analítica.

O objetivo dela é informar o total de venda bruta por cada vendedor (aqui temos cidade vendedora)

In [0]:
tb_name = "gold.olist_total_orders_profit_by_seller_city"
dataset_location = "olist_total_orders_profitprofi_dataset"
target_location = f"dbfs:/FileStore/delta/gold/brazilian_ecommerce/{dataset_location}"

## 1 - Data ingestion


In [0]:
df_order_items = spark.read.table("silver.olist_order_items") # temos o seller_id
df_sellers = spark.read.table("silver.olist_sellers")


## 2 - preparation

In [0]:
df = df_order_items.join(df_sellers, on=['seller_id'], how="inner")

In [0]:
df = df.groupBy("seller_city").agg(
    count("order_id").alias("total_orders_by_seller_city"),
    sum("price").alias("total_items_sell_profit"), # total de arrecadação bruta por produto vendido
    sum("freight_value").alias("total_freight_profit") # total de arrecadação bruta por fretes
).withColumn("total_profit", col("total_items_sell_profit") + col("total_freight_profit")) # total arrecadação bruta

In [0]:
display(df.take(10))


## Saving data

In [0]:
save_dataframe(df, format_mode="delta", table_name=tb_name, target_location=target_location)


## create delta table

In [0]:
create_table(table_name=tb_name, target_location=target_location)

In [0]:
# exit para fechar a execução
dbutils.notebook.exit("OK")

In [0]:
%sql

select * from gold.olist_total_orders_profit_by_seller_city limit 10